<a href="https://colab.research.google.com/github/vindruid/opencv-intro/blob/master/heatmap_town_centre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Video

In [0]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [0]:
# https://drive.google.com/file/d/1sf85EueuGf5VDeHt7UAv2UHPIhEco_4f/view?usp=sharing
file_id = '1sf85EueuGf5VDeHt7UAv2UHPIhEco_4f'
path_video = 'video_town_centre.mp4'
download_file_from_google_drive(file_id, destination)

source: http://www.robots.ox.ac.uk/ActiveVision/Research/Projects/2009bbenfold_headpose/Datasets/TownCentreXVID.avi

we only use a part of whole video

In [0]:
# # wait until the video completely downloaded
# import time
# time.sleep(5)

# Show Video

In [0]:
import os
from IPython.display import HTML
from base64 import b64encode

In [0]:
path_video = os.path.join("video_town_centre.mp4")
mp4 = open(path_video,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Inspect Video

In [0]:
import cv2

In [0]:
path_video = os.path.join("video_town_centre.mp4")
cap  = cv2.VideoCapture(path_video)

_, img = cap.read()

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
count_frame = 1
while img is not None: 
  count_frame += 1 
  _, img = cap.read()

print("TOTAL FRAME:", count_frame)
print("FPS:", fps)
print("Width:", w)
print("Height:", h)

# Heat Map Static (Write to Image)

source: https://github.com/intel-iot-devkit/python-cv-samples/tree/master/examples/motion-heatmap

In [0]:
import numpy as np
import copy

In [0]:
cap = cv2.VideoCapture(path_video)
# pip install opencv-contrib-python
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

# number of frames is a variable for development purposes, you can change the for loop to a while(cap.isOpened()) instead to go through the whole video
num_frames = 350

first_iteration_indicator = 1
for i in range(0, num_frames):
    '''
    There are some important reasons this if statement exists:
        -in the first run there is no previous frame, so this accounts for that
        -the first frame is saved to be used for the overlay after the accumulation has occurred
        -the height and width of the video are used to create an empty image for accumulation (accum_image)
    '''
    if (first_iteration_indicator == 1):
        ret, frame = cap.read()
        first_frame = copy.deepcopy(frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        height, width = gray.shape[:2]
        accum_image = np.zeros((height, width), np.uint8)
        first_iteration_indicator = 0
    else:
        ret, frame = cap.read()  # read a frame
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # convert to grayscale

        fgmask = fgbg.apply(gray)  # remove the background

        # for testing purposes, show the result of the background subtraction
        # cv2.imshow('diff-bkgnd-frame', fgmask)

        # apply a binary threshold only keeping pixels above thresh and setting the result to maxValue.  If you want
        # motion to be picked up more, increase the value of maxValue.  To pick up the least amount of motion over time, set maxValue = 1
        thresh = 2
        maxValue = 2
        ret, th1 = cv2.threshold(fgmask, thresh, maxValue, cv2.THRESH_BINARY)
        # for testing purposes, show the threshold image
        # cv2.imwrite('diff-th1.jpg', th1)

        # add to the accumulated image
        accum_image = cv2.add(accum_image, th1)
        # for testing purposes, show the accumulated image
        # cv2.imwrite('diff-accum.jpg', accum_image)

        # for testing purposes, control frame by frame
        # raw_input("press any key to continue")

# apply a color map
# COLORMAP_PINK also works well, COLORMAP_BONE is acceptable if the background is dark
color_image = im_color = cv2.applyColorMap(accum_image, cv2.COLORMAP_HOT)
# for testing purposes, show the colorMap image
# cv2.imwrite('diff-color.jpg', color_image)

# overlay the color mapped image to the first frame
result_overlay = cv2.addWeighted(first_frame, 0.7, color_image, 0.7, 0)

# save the final overlay image
cv2.imwrite('diff-overlay.jpg', result_overlay)

# cleanup
cap.release()
cv2.destroyAllWindows()

# Heat Map Dinamic (Write to Video)

In [0]:
%%time 
path_video = os.path.join("video_town_centre.mp4")
cap  = cv2.VideoCapture(path_video)

_, img = cap.read()


fps = cap.get(cv2.CAP_PROP_FPS)
W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

path_heatmap = os.path.join('video_heatmap__.mp4') 
vid_writer = cv2.VideoWriter(path_heatmap, cv2.VideoWriter_fourcc(*'MP4V'), fps, (W, H))

fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
accum_image = np.zeros((H, W), np.uint8)
# apply a binary threshold only keeping pixels above thresh and setting the result to maxValue.  If you want
# motion to be picked up more, increase the value of maxValue.  To pick up the least amount of motion over time, set maxValue = 1
thresh = 2
maxValue = 2

count_frame = 1
while img is not None: 


    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fgmask = fgbg.apply(gray)  # remove the background

    _, th1 = cv2.threshold(fgmask, thresh, maxValue, cv2.THRESH_BINARY)

    accum_image = cv2.add(accum_image, th1)

    color_image = cv2.applyColorMap(accum_image, cv2.COLORMAP_HOT)
    result_overlay = cv2.addWeighted(img, 0.7, color_image, 0.7, 0)

    vid_writer.write(result_overlay)

    count_frame += 1 
    _, img = cap.read()

    if count_frame % 100 == 0:
        print("FRAME PROCESSED",count_frame)

vid_writer.release()




# Compress Result Video 

In [0]:
path_compressed_heatmap = 'video_heatmap_c.mp4'
os.system(f"ffmpeg -i {path_heatmap} -vcodec libx264 {path_compressed_heatmap}")


# Show Heatmap

In [0]:
# Show video
mp4 = open(path_compressed_heatmap,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)